In [199]:
import pandas as pd
import numpy as np
import json

from cleanup.cleanup_functions_collec import create_collec_staging
from cleanup.cleanup_siret_functions import clean_numeros

### Upload fichier

In [200]:
path = '../data/datasets/'

In [201]:
communities_df = pd.read_csv(path + 'all_communities_data.csv', delimiter=';', low_memory=False, index_col=False)

### Nettoyage de données

In [202]:
communities_clean = communities_df.copy()

In [203]:
communities_clean = create_collec_staging(communities_clean)

Cleaning duplicates: 36353 entries
After removing duplicates: 36353 entries
drop colummns: 18 columns
 After droping columns : 11 columns
Cleaning duplicates 'nom', 'type', and 'siren': 36353 entries
After removing duplicates: 36347 entries


In [204]:
communities_clean.shape

(36347, 11)

In [205]:
communities_clean[communities_clean['siren'].duplicated(keep=False)]

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50
85,Métropole de Lyon,200046977,DEP,691,NaN,NaN,84,1402326.0,NaN,52.0,True
91,Paris,217500016,DEP,75,NaN,NaN,11,2204773.0,NaN,53.0,True
29389,Paris,217500016,COM,75056,75,075,11.0,2204773.0,200054781,53.0,True
35348,Métropole de Lyon,200046977,MET,NaN,69,069,84,1402326.0,NaN,52.0,True


### Enrichir les données

In [206]:
communities_final = communities_clean.copy()

##### Ajout du code INSEE, qui servira de lien avec les autres sources de données en l'absence du SIREN.
* source georef-france-matching-siren-code.csv, https://public.opendatasoft.com/explore/dataset/georef-france-matching-siren-code/export/?disjunctive.abbr_type

In [207]:
siren_insee = pd.read_csv(path + "georef-france-matching-siren-code.csv", delimiter=';')

In [208]:
siren_insee.duplicated().sum()

0

In [209]:
siren_insee["code_insee"] = siren_insee["Code INSEE Officiel"].apply(clean_numeros)
siren_insee["siren"] = siren_insee["SIREN"].apply(clean_numeros)
siren_insee['type'] = siren_insee["Niveau"]

In [210]:
len('01001')

5

In [211]:
siren_insee[siren_insee["type"]=="COM"]["code_insee"].apply(lambda x: len(x)).value_counts()


code_insee
5    37475
Name: count, dtype: int64

In [212]:
siren_insee[siren_insee["type"]=="DEP"]["code_insee"].apply(lambda x: len(x)).value_counts()

code_insee
2    96
3     6
Name: count, dtype: int64

*Remarque:* Certaines sources n’ont pas le code INSEE des communes sur 5 chiffres. Il faut retirer le 0 devant ici.

In [213]:
mask = (siren_insee['type'] == 'COM') & (siren_insee['code_insee'].str.len() == 5)
siren_insee.loc[mask, 'code_insee'] = siren_insee.loc[mask, 'code_insee'].str.replace(r'^0', '', regex=True)

In [214]:
communities_final = communities_final.merge(siren_insee[['type', 'siren','code_insee']], on=('siren', 'type'), how='left')

In [215]:
communities_final[communities_final["code_insee"].isna()]["type"].value_counts()

type
CC     989
CA     227
MET     22
CU      14
EPT     11
CTU      3
DEP      1
Name: count, dtype: int64

#### 1. Latitude longitude
**source** : selected_communities_data.csv

In [216]:
selected_communities = pd.read_csv(path + 'selected_communities_data.csv', delimiter=';')

In [217]:
selected_communities.head()

,Unnamed: 0,nom,siren,type,cog,cog_3digits,code_departement,code_departement_3digits,code_region,population,epci,url_ptf,url_datagouv,id_datagouv,merge,ptf,trancheeffectifsunitelegale,effectifssup50,longitude,latitude
0,0,Guadeloupe,239710015,REG,1,NaN,NaN,NaN,NaN,396153.0,NaN,https://regionguadeloupe.opendatasoft.com/expl...,NaN,NaN,ptf,KaruData,42.0,True,-61.568686,16.252882
1,1,Île-de-France,237500079,REG,11,NaN,NaN,NaN,NaN,12291279.0,NaN,https://data.iledefrance.fr/explore/?refine.pu...,https://www.data.gouv.fr/fr/organizations/534f...,534fffa8a3a7292c64a780c8,ptf+datagouv,"Ile-de-France, DataGouv",53.0,True,2.504722,48.709167
2,2,Martinique,200055507,CTU,2,NaN,NaN,NaN,NaN,377711.0,NaN,NaN,NaN,NaN,NaN,NaN,51.0,True,-61.015827,14.636792
3,3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,NaN,NaN,2634852.0,NaN,https://data.centrevaldeloire.fr/explore/?sort...,https://www.data.gouv.fr/fr/organizations/regi...,561a7f00c751df4e6dcdbb48,ptf+datagouv,"Centre Val-de-Loire, DataGouv",51.0,True,1.685278,47.480556
4,4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,NaN,NaN,2885864.0,NaN,https://trouver.ternum-bfc.fr/organization/reg...,https://www.data.gouv.fr/fr/organizations/5835...,5835cdb988ee383e0bc65bb3,ptf+datagouv,"Bourgogne-Franche-Comté, DataGouv",51.0,True,4.809167,47.235278


In [218]:
selected_communities["siren"]= selected_communities["siren"].apply(clean_numeros)

In [219]:
communities_final = communities_final.merge(selected_communities[['siren', 'type', 'longitude','latitude']],on=['siren', 'type'], how='left' )

In [220]:
communities_final[communities_final["latitude"].isna()]['type'].value_counts()

type
COM    31842
CC         1
Name: count, dtype: int64

* **Sources** : 
    * 20230823-communes-departement-region.csv,https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25
    

In [221]:
com_dep_reg_df = pd.read_csv(path + '20230823-communes-departement-region.csv')

In [222]:
com_dep_reg_df.shape

(39201, 15)

In [223]:
com_dep_reg_df.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


In [224]:
com_dep_reg_df["code_insee"]=com_dep_reg_df['code_commune_INSEE']

In [225]:
com_dep_reg_df.isna().sum()

code_commune_INSEE          0
nom_commune_postal          0
code_postal                 0
libelle_acheminement        0
ligne_5                 35944
latitude                  269
longitude                 269
code_commune                7
article                 36621
nom_commune                 0
nom_commune_complet         0
code_departement            7
nom_departement           267
code_region               267
nom_region                267
code_insee                  0
dtype: int64

In [226]:

valeurs_dict_la = com_dep_reg_df.groupby('code_insee')['latitude'].first().to_dict()
valeurs_dict_lg = com_dep_reg_df.groupby('code_insee')['longitude'].first().to_dict()
communities_final['latitude'] = communities_final['latitude'].fillna(communities_final['code_insee'].map(valeurs_dict_la))
communities_final['longitude'] =communities_final['longitude'].fillna(communities_final['code_insee'].map(valeurs_dict_lg))


In [227]:
communities_final.isna().sum()

nom                               3
siren                             0
type                              1
cog                            1268
code_dept                       118
code_departement_3digits        118
code_region                      20
population                        2
epci                           1429
trancheeffectifsunitelegale    2109
effectifssup50                    0
code_insee                     1272
longitude                         5
latitude                          5
dtype: int64

In [228]:
communities_final.shape

(36352, 14)

In [229]:
# check vlue null of nom
communities_final[communities_final['siren'].duplicated(keep=False)]

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude
85,Métropole de Lyon,200046977,DEP,691,NaN,NaN,84,1402326.0,NaN,52.0,True,691,4.832011,45.757814
91,Paris,217500016,DEP,75,NaN,NaN,11,2204773.0,NaN,53.0,True,NaN,2.342222,48.856667
10002,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,27676,1.295948,49.201220
10003,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,27058,1.346657,49.235615
29388,Paris,217500016,COM,75056,75,075,11.0,2204773.0,200054781,53.0,True,75056,2.347000,48.859000
35347,Métropole de Lyon,200046977,MET,NaN,69,069,84,1402326.0,NaN,52.0,True,NaN,4.835000,45.758000
35817,Métropole Européenne de Lille,200093201,MET,NaN,59,059,32,24566.0,NaN,NaN,False,NaN,3.045433,50.630992
35818,Métropole Européenne de Lille,200093201,MET,NaN,59,059,32,24566.0,NaN,NaN,False,NaN,3.045433,50.630992
35819,Métropole Européenne de Lille,200093201,MET,NaN,59,059,32,24566.0,NaN,NaN,False,NaN,3.045433,50.630992
35821,CA Agglomération d'Agen,200096956,CA,NaN,47,047,75,5372.0,NaN,NaN,False,NaN,0.631041,44.202304


*Remarque:* création de doublons

In [230]:
communities_final.drop_duplicates(inplace=True)

In [231]:
communities_final[communities_final['siren'].duplicated(keep=False)]

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude
85,Métropole de Lyon,200046977,DEP,691,NaN,NaN,84,1402326.0,NaN,52.0,True,691,4.832011,45.757814
91,Paris,217500016,DEP,75,NaN,NaN,11,2204773.0,NaN,53.0,True,NaN,2.342222,48.856667
10002,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,27676,1.295948,49.201220
10003,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,27058,1.346657,49.235615
29388,Paris,217500016,COM,75056,75,075,11.0,2204773.0,200054781,53.0,True,75056,2.347000,48.859000
35347,Métropole de Lyon,200046977,MET,NaN,69,069,84,1402326.0,NaN,52.0,True,NaN,4.835000,45.758000


*Remarque:* Une seule commune est dupliquée avec un nom nul et des coordonnées (latitude et longitude) différentes.

#### Densite et superficie 
**source**: communes-france-2025.csv

In [232]:
commune_france = pd.read_csv(path + 'communes-france-2025.csv',low_memory=False)

In [233]:
commune_france.columns

Index(['Unnamed: 0', 'code_insee', 'nom_standard', 'nom_sans_pronom', 'nom_a',
       'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'typecom',
       'typecom_texte', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom',
       'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie_code',
       'academie_nom', 'code_postal', 'codes_postaux', 'zone_emploi',
       'code_insee_centre_zone_emploi', 'code_unite_urbaine',
       'nom_unite_urbaine', 'taille_unite_urbaine',
       'type_commune_unite_urbaine', 'statut_commune_unite_urbaine',
       'population', 'superficie_hectare', 'superficie_km2', 'densite',
       'altitude_moyenne', 'altitude_minimale', 'altitude_maximale',
       'latitude_mairie', 'longitude_mairie', 'latitude_centre',
       'longitude_centre', 'grille_densite', 'grille_densite_texte',
       'niveau_equipements_services', 'niveau_equipements_services_texte',
       'gentile', 'url_wikipedia', 'url_villedereve'],
      dtype='object')

In [234]:
commune_france['typecom'].value_counts()

typecom
COM    34935
Name: count, dtype: int64

In [235]:
commune_france.head()

,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.360,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon


In [236]:
commune_france.isna().sum()

Unnamed: 0                               0
code_insee                               0
nom_standard                             0
nom_sans_pronom                          0
nom_a                                    0
nom_de                                   0
nom_sans_accent                          0
nom_standard_majuscule                   0
typecom                                  0
typecom_texte                            0
reg_code                                 0
reg_nom                                  0
dep_code                                 0
dep_nom                                  0
canton_code                             56
canton_nom                              56
epci_code                                3
epci_nom                                 3
academie_code                            0
academie_nom                             0
code_postal                              3
codes_postaux                            3
zone_emploi                              3
code_insee_

In [237]:
commune_france['type']=commune_france['typecom']


In [238]:
mask = (commune_france['type'] == 'COM') & (commune_france['code_insee'].str.len() == 5)
commune_france.loc[mask, 'code_insee'] = commune_france.loc[mask, 'code_insee'].str.replace(r'^0', '', regex=True)

In [239]:
communities_final = communities_final.merge(commune_france[['code_insee', 'type', 'superficie_km2','densite']], on=['code_insee', 'type'], how='left')

In [240]:
communities_final.isna().sum()

nom                               3
siren                             0
type                              1
cog                            1264
code_dept                       118
code_departement_3digits        118
code_region                      20
population                        2
epci                           1425
trancheeffectifsunitelegale    2105
effectifssup50                    0
code_insee                     1268
longitude                         5
latitude                          5
superficie_km2                 1416
densite                        1416
dtype: int64

In [241]:
communities_final.head()

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude,superficie_km2,densite
0,Guadeloupe,239710015,REG,1,NaN,NaN,NaN,396153.0,NaN,42.0,True,01,-61.568686,16.252882,NaN,NaN
1,Île-de-France,237500079,REG,11,NaN,NaN,NaN,12291279.0,NaN,53.0,True,11,2.504722,48.709167,NaN,NaN
2,Martinique,200055507,CTU,2,NaN,NaN,NaN,377711.0,NaN,51.0,True,NaN,-61.015827,14.636792,NaN,NaN
3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,NaN,2634852.0,NaN,51.0,True,24,1.685278,47.480556,NaN,NaN
4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,NaN,2885864.0,NaN,51.0,True,27,4.809167,47.235278,NaN,NaN


#### Compléter les codes départements pour faire le lien  avec les communes ou autres collectivités
source nocodb :ofgl-base-departements-consolidee.csv

In [242]:
bases_departements = pd.read_csv(path +'ofgl-base-departements-consolidee.csv', delimiter=';')

In [243]:
bases_departements['siren'] = bases_departements['siren'].apply(clean_numeros)
bases_departements['dep_code'] = bases_departements['dep_code'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)

In [244]:
valeurs_dict_dep = bases_departements.groupby('siren')['dep_code'].first().to_dict()
communities_final['code_dept'] = communities_final['code_dept'].fillna(communities_final['siren'].map(valeurs_dict_dep))

In [245]:
communities_final[communities_final["code_dept"].isnull()]["type"].value_counts()

type
REG    14
CTU     3
COM     2
Name: count, dtype: int64

In [246]:
communities_final[(communities_final["code_dept"].isnull()) & (communities_final["type"]=='COM')]

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude,superficie_km2,densite
10002,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,27676,1.295948,49.201220,15.0,117.0
10003,NaN,200063782,COM,27676,NaN,NaN,NaN,NaN,200089456,12.0,False,27058,1.346657,49.235615,NaN,NaN


##### Remarque: une seule commune avec nom null et code dept null: siren: 200063782

#### Compléter les codes régions pour faire le lien  avec les communes, départements ou autres collectivités
Sources: NocoDB ofgl-base-regions-consolidee.csv 

In [247]:
communities_final[communities_final['code_region'].isna()]["type"].value_counts()

type
REG    14
CTU     3
COM     2
Name: count, dtype: int64

In [248]:
base_regions = pd.read_csv(path + 'ofgl-base-regions-consolidee.csv', delimiter=';')

In [249]:
base_regions['siren'] = base_regions['siren'].apply(clean_numeros)
base_regions['reg_code'] = base_regions['reg_code'].apply(lambda x: str(x).strip().upper() if pd.notna(x) else x)

In [250]:
valeurs_dict_reg = base_regions.groupby('siren')['reg_code'].first().to_dict()
communities_final['code_region'] = communities_final['code_region'].fillna(communities_final['siren'].map(valeurs_dict_reg))

In [251]:
communities_final[communities_final['code_region'].isna()]["type"].value_counts()

type
COM    2
Name: count, dtype: int64

Remarque: la même commune avec code région null: 200063782

#### Les élus

##### Ajout elus minicipaux
* source: nocodb: elus-conseillers-municipaux-cm.csv

In [252]:
elus_minicip = pd.read_csv(path + "elus-conseillers-municipaux-cm.csv", delimiter=';', low_memory=False)

In [253]:
elus_minicip.shape

(488719, 16)

In [254]:
elus_minicip.columns

Index(['Code du département', 'Libellé du département',
       'Code de la collectivité à statut particulier',
       'Libellé de la collectivité à statut particulier', 'Code de la commune',
       'Libellé de la commune', 'Nom de l'élu', 'Prénom de l'élu', 'Code sexe',
       'Date de naissance', 'Code de la catégorie socio-professionnelle',
       'Libellé de la catégorie socio-professionnelle',
       'Date de début du mandat', 'Libellé de la fonction',
       'Date de début de la fonction', 'Code nationalité'],
      dtype='object')

In [255]:
elus_minicip.isna().sum()

Code du département                                  3315
Libellé du département                               3315
Code de la collectivité à statut particulier       481236
Libellé de la collectivité à statut particulier    481236
Code de la commune                                      0
Libellé de la commune                                   0
Nom de l'élu                                            1
Prénom de l'élu                                         0
Code sexe                                               0
Date de naissance                                       0
Code de la catégorie socio-professionnelle            142
Libellé de la catégorie socio-professionnelle           0
Date de début du mandat                                 0
Libellé de la fonction                             338034
Date de début de la fonction                       338034
Code nationalité                                        0
dtype: int64

In [256]:
elus_minicip["Libellé de la fonction"].value_counts()

Libellé de la fonction
Maire                     34844
1er adjoint au Maire      33863
2ème adjoint au Maire     31238
3ème adjoint au Maire     19969
4ème adjoint au Maire     11000
5ème adjoint au Maire      5839
6ème adjoint au Maire      3820
7ème adjoint au Maire      2826
Maire délégué              2286
8ème adjoint au Maire      2215
9ème adjoint au Maire       917
10ème adjoint au Maire      466
11ème adjoint au Maire      326
12ème adjoint au Maire      246
13ème adjoint au Maire      193
14ème adjoint au Maire      148
15ème adjoint au Maire      104
16ème adjoint au Maire       92
17ème adjoint au Maire       71
18ème adjoint au Maire       57
19ème adjoint au Maire       42
20ème adjoint au Maire       37
21ème adjoint au Maire       29
22ème adjoint au Maire       12
25ème adjoint au Maire        8
24ème adjoint au Maire        8
23ème adjoint au Maire        8
26ème adjoint au Maire        5
29ème adjoint au Maire        2
37ème adjoint au Maire        2
36ème adjoint au 

In [257]:
elus_minicip_maire = elus_minicip[elus_minicip["Libellé de la fonction"] == "Maire"]

In [258]:
elus_minicip_maire["code_insee"] = elus_minicip_maire["Code de la commune"]
elus_minicip_maire['type_elu'] = "Maire"
elus_minicip_maire['nom_elu'] = elus_minicip_maire["Nom de l'élu"]
elus_minicip_maire['type'] = "COM"

/tmp/ipykernel_62332/2562459737.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elus_minicip_maire["code_insee"] = elus_minicip_maire["Code de la commune"]
/tmp/ipykernel_62332/2562459737.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elus_minicip_maire['type_elu'] = "Maire"
/tmp/ipykernel_62332/2562459737.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [259]:
elus_minicip_maire.head()

,Code du département,Libellé du département,Code de la collectivité à statut particulier,Libellé de la collectivité à statut particulier,Code de la commune,Libellé de la commune,Nom de l'élu,Prénom de l'élu,Code sexe,Date de naissance,Code de la catégorie socio-professionnelle,Libellé de la catégorie socio-professionnelle,Date de début du mandat,Libellé de la fonction,Date de début de la fonction,Code nationalité,code_insee,type_elu,nom_elu,type
3,1,Ain,NaN,NaN,1001,L'Abergement-Clémenciat,BOULON,Daniel,M,04/03/1951,74.0,Ancien cadre,18/05/2020,Maire,26/05/2020,FR,1001,Maire,BOULON,COM
21,1,Ain,NaN,NaN,1002,L'Abergement-De-Varey,ORSET,Max,M,02/11/1947,65.0,"Ouvrier qualifié de la manutention, du magasin...",18/05/2020,Maire,27/05/2020,FR,1002,Maire,ORSET,COM
37,1,Ain,NaN,NaN,1004,Ambérieu-En-Bugey,FABRE,Daniel,M,07/09/1961,74.0,Ancien cadre,18/05/2020,Maire,28/05/2020,FR,1004,Maire,FABRE,COM
60,1,Ain,NaN,NaN,1005,Ambérieux-En-Dombes,FORNES,Christine,F,06/07/1962,21.0,Artisan,02/05/2021,Maire,07/05/2021,FR,1005,Maire,FORNES,COM
77,1,Ain,NaN,NaN,1006,Ambléon,BIONDA,Annie,F,28/11/1951,77.0,Ancien employé,18/05/2020,Maire,25/05/2020,FR,1006,Maire,BIONDA,COM


In [260]:
communities_final = communities_final.merge(elus_minicip_maire[['code_insee','nom_elu','type_elu', 'type']], on=('code_insee','type'), how='left')

In [261]:

communities_final[communities_final['type_elu'].notna()]['type'].value_counts()

type
COM    34758
Name: count, dtype: int64

##### Ajout elus departementaux
* source: nocodb: elus-conseillers-departementaux-cd.csv

In [262]:
elus_dept = pd.read_csv(path + "elus-conseillers-departementaux-cd.csv", delimiter=';')


In [263]:
elus_dept.head()

,Code du département,Libellé du département,Code du canton,Libellé du canton,Nom de l'élu,Prénom de l'élu,Code sexe,Date de naissance,Code de la catégorie socio-professionnelle,Libellé de la catégorie socio-professionnelle,Date de début du mandat,Libellé de la fonction,Date de début de la fonction
0,01,Ain,101,Ambérieu-En-Bugey,BRUNET,Joël,M,02/05/1955,23,Chef d'entreprise de 10 salariés ou plus,01/07/2021,NaN,NaN
1,01,Ain,101,Ambérieu-En-Bugey,PETIT,Aurélie,F,29/08/1982,38,Ingénieur et cadre technique d'entreprise,01/07/2021,NaN,NaN
2,01,Ain,102,Attignat,FOURNIER,Clotilde,F,01/09/1966,35,"Profession de l'information, des arts et des s...",01/07/2021,9ème Vice-président du conseil départemental,01/07/2021
3,01,Ain,102,Attignat,MARTIN,Walter,M,07/05/1966,38,Ingénieur et cadre technique d'entreprise,01/07/2021,NaN,NaN
4,01,Ain,103,Valserhône,LARMANJAT,Guy,M,28/09/1962,38,Ingénieur et cadre technique d'entreprise,01/07/2021,NaN,NaN


In [264]:
elus_dept["Libellé de la fonction"].value_counts()

Libellé de la fonction
Président du conseil départemental                  95
4ème Vice-président du conseil départemental        94
6ème Vice-président du conseil départemental        94
2ème Vice-président du conseil départemental        93
1er Vice-président du conseil départemental         92
7ème Vice-président du conseil départemental        92
3ème Vice-président du conseil départemental        91
5ème Vice-président du conseil départemental        91
8ème Vice-président du conseil départemental        87
9ème Vice-président du conseil départemental        80
10ème Vice-président du conseil départemental       71
11ème Vice-président du conseil départemental       54
12ème Vice-président du conseil départemental       46
13ème Vice-président du conseil départemental       28
14ème Vice-président du conseil départemental       20
15ème Vice-président du conseil départemental       19
Président de commission du conseil départemental    17
Name: count, dtype: int64

In [265]:
elus_dept_president = elus_dept[elus_dept["Libellé de la fonction"] == "Président du conseil départemental"]

In [266]:
elus_dept_president['type_elu'] = "Président du conseil départemental"
elus_dept_president['nom_elu'] = elus_dept["Nom de l'élu"]
elus_dept_president["type"] = "DEP"
elus_dept_president["code_dept"] =  elus_dept_president["Code du département"]

/tmp/ipykernel_62332/3650462116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elus_dept_president['type_elu'] = "Président du conseil départemental"
/tmp/ipykernel_62332/3650462116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elus_dept_president['nom_elu'] = elus_dept["Nom de l'élu"]
/tmp/ipykernel_62332/3650462116.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [267]:
valeurs_dict_nomelu_dept = elus_dept_president.groupby('code_dept')['nom_elu'].first().to_dict()
communities_final.loc[communities_final['type'] == 'DEP', 'nom_elu'] = (
    communities_final.loc[communities_final['type'] == 'DEP', 'nom_elu']
    .fillna(communities_final['code_dept'].map(valeurs_dict_nomelu_dept))
)
communities_final.loc[communities_final['type'] == 'DEP', 'type_elu'] = (
    communities_final.loc[communities_final['type'] == 'DEP', 'type_elu']
    .fillna("Président du conseil départemental")
)

In [268]:
communities_final[communities_final['type']=='DEP']

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude,superficie_km2,densite,nom_elu,type_elu
17,Ain,220100010,DEP,01,01,NaN,84,659180.0,NaN,51.0,True,01,5.348889,46.099444,NaN,NaN,DEGUERRY,Président du conseil départemental
18,Aisne,220200026,DEP,02,02,NaN,32,546527.0,NaN,51.0,True,02,3.558333,49.559444,NaN,NaN,FRICOTEAUX,Président du conseil départemental
19,Allier,220300016,DEP,03,03,NaN,84,347035.0,NaN,42.0,True,03,3.188333,46.393611,NaN,NaN,RIBOULET,Président du conseil départemental
20,Alpes-de-Haute-Provence,220400014,DEP,04,04,NaN,93,168381.0,NaN,42.0,True,04,6.243889,44.106111,NaN,NaN,BARREILLE,Président du conseil départemental
21,Hautes-Alpes,220500011,DEP,05,05,NaN,93,145883.0,NaN,42.0,True,05,6.263056,44.663611,NaN,NaN,BERNARD,Président du conseil départemental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Val-de-Marne,229400288,DEP,94,94,NaN,11,1397035.0,NaN,52.0,True,94,2.468889,48.777500,NaN,NaN,CAPITANIO,Président du conseil départemental
111,Val-d'Oise,229501275,DEP,95,95,NaN,11,1239262.0,NaN,51.0,True,95,2.131111,49.082778,NaN,NaN,CAVECCHI,Président du conseil départemental
112,Guadeloupe,229710017,DEP,971,971,NaN,1,396153.0,NaN,51.0,True,971,-61.568686,16.252882,NaN,NaN,LOSBAR,Président du conseil départemental
113,La Réunion,229740014,DEP,974,974,NaN,4,863063.0,NaN,52.0,True,974,-21.114533,55.532062,NaN,NaN,MELCHIOR,Président du conseil départemental


##### Ajout elus regionaux
* Source: nocodb: elus-conseillers-regionaux-cr.csv

In [269]:
elus_reg = pd.read_csv(path + 'elus-conseillers-regionaux-cr.csv', delimiter=';')
elus_reg

,Code de la région,Libellé de la région,Code de la section départementale,Libellé de la section départementale,Nom de l'élu,Prénom de l'élu,Code sexe,Date de naissance,Code de la catégorie socio-professionnelle,Libellé de la catégorie socio-professionnelle,Date de début du mandat,Libellé de la fonction,Date de début de la fonction
0,1,Guadeloupe,971,Guadeloupe,ARMOUGOM,Betty Véronique,F,09/07/1965,23.0,Chef d'entreprise de 10 salariés ou plus,02/07/2021,NaN,NaN
1,1,Guadeloupe,971,Guadeloupe,BAILLET,Patricia,F,17/06/1970,37.0,Cadre administratif et commercial d'entreprise,02/07/2021,NaN,NaN
2,1,Guadeloupe,971,Guadeloupe,BARDAIL,Jean,M,30/05/1951,75.0,Ancienne profession intermédiaire,02/07/2021,3ème Vice-président du conseil régional,02/07/2021
3,1,Guadeloupe,971,Guadeloupe,BITUFWILA,Aurélie,F,21/08/1991,42.0,"Professeur des écoles, instituteur et assimilé",02/07/2021,NaN,NaN
4,1,Guadeloupe,971,Guadeloupe,BONDOT-GALAS,Gersiane,F,09/02/1963,42.0,"Professeur des écoles, instituteur et assimilé",02/07/2021,8ème Vice-président du conseil régional,02/07/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1744,93,Provence-Alpes-Cote D'Azur,84,Vaucluse,PERILHOU,Jean-François,M,05/09/1982,31.0,Profession libérale,02/07/2021,NaN,NaN
1745,93,Provence-Alpes-Cote D'Azur,84,Vaucluse,RIGAULT,Anne-Sophie,F,23/03/1976,85.0,Personne diverse sans activité professionnelle...,02/07/2021,NaN,NaN
1746,93,Provence-Alpes-Cote D'Azur,84,Vaucluse,RIMBERT,Catherine,F,13/09/1968,31.0,Profession libérale,02/07/2021,NaN,NaN
1747,93,Provence-Alpes-Cote D'Azur,84,Vaucluse,RIPERT,Gilles,M,04/06/1958,31.0,Profession libérale,12/05/2022,NaN,NaN


In [270]:
communities_final[communities_final['type']=='DEP']['code_region']

17     84
18     32
19     84
20     93
21     93
       ..
110    11
111    11
112     1
113     4
114     6
Name: code_region, Length: 98, dtype: object

In [271]:
elus_reg["Libellé de la fonction"].value_counts()

Libellé de la fonction
3ème Vice-président du conseil régional     14
8ème Vice-président du conseil régional     14
Président du conseil régional               14
4ème Vice-président du conseil régional     14
2ème Vice-président du conseil régional     14
1er Vice-président du conseil régional      13
6ème Vice-président du conseil régional     13
9ème Vice-président du conseil régional     13
7ème Vice-président du conseil régional     12
5ème Vice-président du conseil régional     12
10ème Vice-président du conseil régional    12
12ème Vice-président du conseil régional    12
11ème Vice-président du conseil régional    12
13ème Vice-président du conseil régional    12
15ème Vice-président du conseil régional    10
14ème Vice-président du conseil régional     9
Name: count, dtype: int64

In [272]:
elus_reg_president = elus_reg[elus_reg["Libellé de la fonction"]=="Président du conseil régional"]

In [273]:
elus_reg_president['type_elu'] = "Président du conseil régional"
elus_reg_president['nom_elu'] = elus_reg["Nom de l'élu"]
elus_reg_president["type"] = "REG"
elus_reg_president["code_region"] =  elus_reg_president["Code de la région"].apply(lambda x: str(x).strip() if pd.notna(x) else x)

/tmp/ipykernel_62332/2490365004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elus_reg_president['type_elu'] = "Président du conseil régional"
/tmp/ipykernel_62332/2490365004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elus_reg_president['nom_elu'] = elus_reg["Nom de l'élu"]
/tmp/ipykernel_62332/2490365004.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [274]:
valeurs_dict_nomelu_reg = elus_reg_president.groupby('code_region')['nom_elu'].first().to_dict()

# Remplacer les valeurs NaN dans 'nom_elu' pour les régions
communities_final.loc[communities_final['type'] == 'REG', 'nom_elu'] = (
    communities_final.loc[communities_final['type'] == 'REG', 'nom_elu']
    .fillna(communities_final['code_region'].map(valeurs_dict_nomelu_reg))
)
communities_final.loc[communities_final['type'] == 'REG', 'type_elu'] = (
    communities_final.loc[communities_final['type'] == 'REG', 'type_elu']
    .fillna("Président du conseil régional")
)

In [275]:
valeurs_dict_nomelu_reg

{'1': 'CHALUS',
 '11': 'PÉCRESSE',
 '24': 'BONNEAU',
 '27': 'DUFAY',
 '28': 'MORIN',
 '32': 'BERTRAND',
 '4': 'BELLO',
 '44': 'LEROY',
 '52': 'MORANÇAIS',
 '53': 'CHESNAIS-GIRARD',
 '75': 'ROUSSET',
 '76': 'DELGA',
 '84': 'PANNEKOUCKE',
 '93': 'MUSELIER'}

In [276]:
communities_final[communities_final["type"]=="REG"]

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude,superficie_km2,densite,nom_elu,type_elu
0,Guadeloupe,239710015,REG,1,NaN,NaN,1,396153.0,NaN,42.0,True,01,-61.568686,16.252882,NaN,NaN,CHALUS,Président du conseil régional
1,Île-de-France,237500079,REG,11,NaN,NaN,11,12291279.0,NaN,53.0,True,11,2.504722,48.709167,NaN,NaN,PÉCRESSE,Président du conseil régional
3,Centre-Val de Loire,234500023,REG,24,NaN,NaN,24,2634852.0,NaN,51.0,True,24,1.685278,47.480556,NaN,NaN,BONNEAU,Président du conseil régional
4,Bourgogne-Franche-Comté,200053726,REG,27,NaN,NaN,27,2885864.0,NaN,51.0,True,27,4.809167,47.235278,NaN,NaN,DUFAY,Président du conseil régional
5,Normandie,200053403,REG,28,NaN,NaN,28,3403309.0,NaN,52.0,True,28,0.106667,49.121111,NaN,NaN,MORIN,Président du conseil régional
7,Hauts-de-France,200053742,REG,32,NaN,NaN,32,6096177.0,NaN,52.0,True,32,2.775278,49.966111,NaN,NaN,BERTRAND,Président du conseil régional
8,La Réunion,239740012,REG,4,NaN,NaN,4,863063.0,NaN,51.0,True,04,-21.114533,55.532062,NaN,NaN,BELLO,Président du conseil régional
9,Grand Est,200052264,REG,44,NaN,NaN,44,5657093.0,NaN,52.0,True,44,5.619444,48.689167,NaN,NaN,LEROY,Président du conseil régional
10,Pays de la Loire,234400034,REG,52,NaN,NaN,52,3846161.0,NaN,51.0,True,52,0.823889,47.474722,NaN,NaN,MORANÇAIS,Président du conseil régional
11,Bretagne,233500016,REG,53,NaN,NaN,53,3407564.0,NaN,51.0,True,53,-1.161389,48.179722,NaN,NaN,CHESNAIS-GIRARD,Président du conseil régional


#### Obligation publication

In [277]:
## TODO

In [278]:
communities_final[communities_final['type']=='COM']

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude,superficie_km2,densite,nom_elu,type_elu
115,L'Abergement-Clémenciat,210100012,COM,01001,01,001,84.0,794.0,200069193,11.0,False,1001,4.926114,46.153426,16.0,53.0,BOULON,Maire
116,L'Abergement-de-Varey,210100020,COM,01002,01,001,84.0,249.0,240100883,2.0,False,1002,5.428017,46.009188,9.0,29.0,ORSET,Maire
117,Ambérieu-en-Bugey,210100046,COM,01004,01,001,84.0,14428.0,240100883,32.0,True,1004,5.365228,45.959530,24.0,607.0,FABRE,Maire
118,Ambérieux-en-Dombes,210100053,COM,01005,01,001,84.0,1723.0,200042497,12.0,False,1005,4.912273,45.996180,16.0,118.0,FORNES,Maire
119,Ambléon,210100061,COM,01006,01,001,84.0,117.0,200040350,1.0,False,1006,5.594320,45.749499,6.0,19.0,BIONDA,Maire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35079,M'Tsangamouji,200008829,COM,97613,976,976,6.0,6586.0,200059871,NaN,False,97613,45.087170,-12.751310,22.0,298.0,IBRAHIMA,Maire
35080,Ouangani,200008852,COM,97614,976,976,6.0,10393.0,200059871,NaN,False,97614,45.137910,-12.837096,18.0,558.0,AMBDI,Maire
35081,Pamandzi,200008860,COM,97615,976,976,6.0,11802.0,200050532,NaN,False,97615,45.284206,-12.796135,4.0,2686.0,MADI SOUF,Maire
35082,Sada,200008878,COM,97616,976,976,6.0,11619.0,200059871,NaN,False,97616,45.118550,-12.861165,11.0,1028.0,ABDALLAH,Maire


In [279]:
## remetre le 0 pour le code_insee des communes
mask = (communities_final['type'] == 'COM') & (communities_final['code_insee'].str.len() < 5)
communities_final.loc[mask, 'code_insee'] = communities_final.loc[mask, 'code_insee'].str.zfill(5)

In [280]:
communities_final[communities_final['type']=='COM']

,nom,siren,type,cog,code_dept,code_departement_3digits,code_region,population,epci,trancheeffectifsunitelegale,effectifssup50,code_insee,longitude,latitude,superficie_km2,densite,nom_elu,type_elu
115,L'Abergement-Clémenciat,210100012,COM,01001,01,001,84.0,794.0,200069193,11.0,False,01001,4.926114,46.153426,16.0,53.0,BOULON,Maire
116,L'Abergement-de-Varey,210100020,COM,01002,01,001,84.0,249.0,240100883,2.0,False,01002,5.428017,46.009188,9.0,29.0,ORSET,Maire
117,Ambérieu-en-Bugey,210100046,COM,01004,01,001,84.0,14428.0,240100883,32.0,True,01004,5.365228,45.959530,24.0,607.0,FABRE,Maire
118,Ambérieux-en-Dombes,210100053,COM,01005,01,001,84.0,1723.0,200042497,12.0,False,01005,4.912273,45.996180,16.0,118.0,FORNES,Maire
119,Ambléon,210100061,COM,01006,01,001,84.0,117.0,200040350,1.0,False,01006,5.594320,45.749499,6.0,19.0,BIONDA,Maire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35079,M'Tsangamouji,200008829,COM,97613,976,976,6.0,6586.0,200059871,NaN,False,97613,45.087170,-12.751310,22.0,298.0,IBRAHIMA,Maire
35080,Ouangani,200008852,COM,97614,976,976,6.0,10393.0,200059871,NaN,False,97614,45.137910,-12.837096,18.0,558.0,AMBDI,Maire
35081,Pamandzi,200008860,COM,97615,976,976,6.0,11802.0,200050532,NaN,False,97615,45.284206,-12.796135,4.0,2686.0,MADI SOUF,Maire
35082,Sada,200008878,COM,97616,976,976,6.0,11619.0,200059871,NaN,False,97616,45.118550,-12.861165,11.0,1028.0,ABDALLAH,Maire
